# Auto-Correct System

We are going to implement an auto-correct system that is very effective and useful.

<a name='0'></a>
## About

We use autocorrect every day on your cell phone and computer. In this project, we will build a model that is able to predict the probability of occurance of the particular word in any sentence. The model we are about to implement is not identical to the one used in our phones, but it is still very robust and effective. 

In this project we will: 

- Get a word count given a corpus
- Get a word probability in the corpus 
- Manipulate strings 
- Filter strings 
- Implement Minimum edit distance to compare strings and to help find the optimal path for the edits. 
- Understand how dynamic programming works
 
- For example, if we type in the wrong word in this particular sentence **"I am going to the grocery ship"**. Here, we meant to write **"I am going to the grocery shop"**.

- So the model will be able to predict the word occurance and input the correct word in the sentence.

<a name='1'></a>
# Data Preprocessing 

In [1]:
import re
from collections import Counter
import numpy as np
import pandas as pd

In [2]:
def process_data(file_name):
    words = []

    with open(file_name) as f:
        file_name_d = f.read()
    
    file_name_d = file_name_d.lower()
    words = re.findall('\w+', file_name_d)
    
    return words

In [3]:
word_l = process_data('shakespeare.txt')
vocab = set(word_l)

print(f"The first ten words in the text are: \n{word_l[0:20]}")
print(f"There are {len(vocab)} unique words in the vocabulary.")

The first ten words in the text are: 
['o', 'for', 'a', 'muse', 'of', 'fire', 'that', 'would', 'ascend', 'the', 'brightest', 'heaven', 'of', 'invention', 'a', 'kingdom', 'for', 'a', 'stage', 'princes']
There are 6116 unique words in the vocabulary.


In [4]:
def get_count(word_l):
    word_count_dict = {}
    word_count_dict = Counter(word_l)   

    return word_count_dict

In [33]:
word_count_dict = get_count(word_l)
print(f"There are {len(word_count_dict)} key values pairs")
print(f"The count for the word 'thou' is {word_count_dict.get('thou',0)}")

There are 6116 key values pairs
The count for the word 'thou' is 334


In [6]:
def get_probs(word_count_dict):
    probs = {}

    m = sum(word_count_dict.values())
    
    for keys in word_count_dict.keys():
        probs[keys] = word_count_dict[keys] / m

    return probs

In [34]:
probs = get_probs(word_count_dict)
print(f"Length of probs is {len(probs)}")
print(f"P('thou') is {probs['thou']:.4f}")

Length of probs is 6116
P('thou') is 0.0062


In [8]:
def delete_letter(word, verbose=False):
    delete_l = []
    split_l = []
    
    for c in range(len(word)):
            split_l.append((word[:c], word[c:]))
            
    for a,b in split_l :
        delete_l.append(a + b[1:])

    if verbose: 
        print(f"input word {word}, \nsplit_l = {split_l}, \ndelete_l = {delete_l}")

    return delete_l

In [35]:
delete_word_l = delete_letter(word="core", verbose=True)

input word core, 
split_l = [('', 'core'), ('c', 'ore'), ('co', 're'), ('cor', 'e')], 
delete_l = ['ore', 'cre', 'coe', 'cor']


In [10]:
print(f"Number of outputs of delete_letter('at') is {len(delete_letter('at'))}")

Number of outputs of delete_letter('at') is 2


In [11]:
def switch_letter(word, verbose=False):
    switch_l = []
    split_l = []
    len_word = len(word)

    for c in range(len_word):
        split_l.append((word[:c], word[c:]))
        
    switch_l = [a + b[1] + b[0] + b[2:] for a,b in split_l if len(b) >= 2]

    
    if verbose: 
        print(f"Input word = {word} \nsplit_l = {split_l} \nswitch_l = {switch_l}") 

    return switch_l

In [12]:
switch_word_l = switch_letter(word="eta", verbose=True)

Input word = eta 
split_l = [('', 'eta'), ('e', 'ta'), ('et', 'a')] 
switch_l = ['tea', 'eat']


In [13]:
print(f"Number of outputs of switch_letter('at') is {len(switch_letter('at'))}")

Number of outputs of switch_letter('at') is 1


In [14]:
def replace_letter(word, verbose=False):    
    letters = 'abcdefghijklmnopqrstuvwxyz'
    replace_l = []
    split_l = []
    
    for c in range(len(word)):
        split_l.append((word[0:c],word[c:]))
    
    replace_l = [a + l + (b[1:] if len(b)> 1 else '') for a,b in split_l if b for l in letters]
    replace_set=set(replace_l)
    replace_set.remove(word)
    replace_l = sorted(list(replace_set))
    
    if verbose: 
        print(f"Input word = {word} \nsplit_l = {split_l} \nreplace_l {replace_l}")   
    
    return replace_l

In [15]:
replace_l = replace_letter(word='can', verbose=True)

Input word = can 
split_l = [('', 'can'), ('c', 'an'), ('ca', 'n')] 
replace_l ['aan', 'ban', 'caa', 'cab', 'cac', 'cad', 'cae', 'caf', 'cag', 'cah', 'cai', 'caj', 'cak', 'cal', 'cam', 'cao', 'cap', 'caq', 'car', 'cas', 'cat', 'cau', 'cav', 'caw', 'cax', 'cay', 'caz', 'cbn', 'ccn', 'cdn', 'cen', 'cfn', 'cgn', 'chn', 'cin', 'cjn', 'ckn', 'cln', 'cmn', 'cnn', 'con', 'cpn', 'cqn', 'crn', 'csn', 'ctn', 'cun', 'cvn', 'cwn', 'cxn', 'cyn', 'czn', 'dan', 'ean', 'fan', 'gan', 'han', 'ian', 'jan', 'kan', 'lan', 'man', 'nan', 'oan', 'pan', 'qan', 'ran', 'san', 'tan', 'uan', 'van', 'wan', 'xan', 'yan', 'zan']


In [16]:
print(f"Number of outputs of switch_letter('at') is {len(switch_letter('at'))}")

Number of outputs of switch_letter('at') is 1


In [17]:
def insert_letter(word, verbose=False):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    insert_l = []
    split_l = []

    for c in range(len(word)+1):
        split_l.append((word[0:c],word[c:]))
        
    insert_l = [ a + l + b for a,b in split_l for l in letters]

    if verbose: 
        print(f"Input word {word} \nsplit_l = {split_l} \ninsert_l = {insert_l}")
    
    return insert_l

In [18]:
insert_l = insert_letter('at', True)
print(f"Number of strings output by insert_letter('at') is {len(insert_l)}")

Input word at 
split_l = [('', 'at'), ('a', 't'), ('at', '')] 
insert_l = ['aat', 'bat', 'cat', 'dat', 'eat', 'fat', 'gat', 'hat', 'iat', 'jat', 'kat', 'lat', 'mat', 'nat', 'oat', 'pat', 'qat', 'rat', 'sat', 'tat', 'uat', 'vat', 'wat', 'xat', 'yat', 'zat', 'aat', 'abt', 'act', 'adt', 'aet', 'aft', 'agt', 'aht', 'ait', 'ajt', 'akt', 'alt', 'amt', 'ant', 'aot', 'apt', 'aqt', 'art', 'ast', 'att', 'aut', 'avt', 'awt', 'axt', 'ayt', 'azt', 'ata', 'atb', 'atc', 'atd', 'ate', 'atf', 'atg', 'ath', 'ati', 'atj', 'atk', 'atl', 'atm', 'atn', 'ato', 'atp', 'atq', 'atr', 'ats', 'att', 'atu', 'atv', 'atw', 'atx', 'aty', 'atz']
Number of strings output by insert_letter('at') is 78


In [19]:
print(f"Number of outputs of insert_letter('at') is {len(insert_letter('at'))}")

Number of outputs of insert_letter('at') is 78


In [20]:
def edit_one_letter(word, allow_switches = True):
    edit_one_set = set()
    edit_one_set.update(delete_letter(word))
    
    if allow_switches:
        edit_one_set.update(switch_letter(word))
        
    edit_one_set.update(replace_letter(word))
    edit_one_set.update(insert_letter(word))

    return edit_one_set

In [21]:
tmp_word = "at"
tmp_edit_one_set = edit_one_letter(tmp_word)
tmp_edit_one_l = sorted(list(tmp_edit_one_set))

print(f"input word {tmp_word} \nedit_one_l \n{tmp_edit_one_l}\n")
print(f"The type of the returned object should be a set {type(tmp_edit_one_set)}")
print(f"Number of outputs from edit_one_letter('at') is {len(edit_one_letter('at'))}")

input word at 
edit_one_l 
['a', 'aa', 'aat', 'ab', 'abt', 'ac', 'act', 'ad', 'adt', 'ae', 'aet', 'af', 'aft', 'ag', 'agt', 'ah', 'aht', 'ai', 'ait', 'aj', 'ajt', 'ak', 'akt', 'al', 'alt', 'am', 'amt', 'an', 'ant', 'ao', 'aot', 'ap', 'apt', 'aq', 'aqt', 'ar', 'art', 'as', 'ast', 'ata', 'atb', 'atc', 'atd', 'ate', 'atf', 'atg', 'ath', 'ati', 'atj', 'atk', 'atl', 'atm', 'atn', 'ato', 'atp', 'atq', 'atr', 'ats', 'att', 'atu', 'atv', 'atw', 'atx', 'aty', 'atz', 'au', 'aut', 'av', 'avt', 'aw', 'awt', 'ax', 'axt', 'ay', 'ayt', 'az', 'azt', 'bat', 'bt', 'cat', 'ct', 'dat', 'dt', 'eat', 'et', 'fat', 'ft', 'gat', 'gt', 'hat', 'ht', 'iat', 'it', 'jat', 'jt', 'kat', 'kt', 'lat', 'lt', 'mat', 'mt', 'nat', 'nt', 'oat', 'ot', 'pat', 'pt', 'qat', 'qt', 'rat', 'rt', 'sat', 'st', 't', 'ta', 'tat', 'tt', 'uat', 'ut', 'vat', 'vt', 'wat', 'wt', 'xat', 'xt', 'yat', 'yt', 'zat', 'zt']

The type of the returned object should be a set <class 'set'>
Number of outputs from edit_one_letter('at') is 129


In [22]:
def edit_two_letters(word, allow_switches = True):
    edit_two_set = set()
    edit_one = edit_one_letter(word, allow_switches = allow_switches)
    
    for w in edit_one:
        if w:
            edit_two = edit_one_letter(w,allow_switches=allow_switches)
            edit_two_set.update(edit_two)
    
    return edit_two_set

In [23]:
tmp_edit_two_set = edit_two_letters("a")
tmp_edit_two_l = sorted(list(tmp_edit_two_set))
print(f"Number of strings with edit distance of two: {len(tmp_edit_two_l)}")
print(f"First 10 strings {tmp_edit_two_l[:10]}")
print(f"Last 10 strings {tmp_edit_two_l[-10:]}")
print(f"The data type of the returned object should be a set {type(tmp_edit_two_set)}")
print(f"Number of strings that are 2 edit distances from 'at' is {len(edit_two_letters('at'))}")

Number of strings with edit distance of two: 2654
First 10 strings ['', 'a', 'aa', 'aaa', 'aab', 'aac', 'aad', 'aae', 'aaf', 'aag']
Last 10 strings ['zv', 'zva', 'zw', 'zwa', 'zx', 'zxa', 'zy', 'zya', 'zz', 'zza']
The data type of the returned object should be a set <class 'set'>
Number of strings that are 2 edit distances from 'at' is 7154


In [24]:
print( [] and ["a","b"] )
print( [] or ["a","b"] )
val1 =  ["Most","Likely"] or ["Less","so"] or ["least","of","all"] 
print(val1)
val2 =  [] or [] or ["least","of","all"]
print(val2)

[]
['a', 'b']
['Most', 'Likely']
['least', 'of', 'all']


In [25]:
def get_corrections(word, probs, vocab, n=2, verbose = False):
    suggestions = []
    n_best = []
    
    suggestions = list((word in vocab and word) or edit_one_letter(word).intersection(vocab) or edit_two_letters(word).intersection(vocab))
    n_best = [[s, probs[s]] for s in list(reversed(suggestions))]
    
    if verbose: 
        print("entered word = ", word, "\nsuggestions = ", suggestions)

    return n_best

In [26]:
my_word = 'dys' 
tmp_corrections = get_corrections(my_word, probs, vocab, 2, verbose=True)
for i, word_prob in enumerate(tmp_corrections):
    print(f"word {i}: {word_prob[0]}, probability {word_prob[1]:.6f}")

print(f"data type of corrections {type(tmp_corrections)}")

entered word =  dys 
suggestions =  ['days', 'dye']
word 0: dye, probability 0.000019
word 1: days, probability 0.000410
data type of corrections <class 'list'>


# Minimum Edit distance


In [27]:
def min_edit_distance(source, target, ins_cost = 1, del_cost = 1, rep_cost = 2):
    m = len(source) 
    n = len(target) 

    D = np.zeros((m+1, n+1), dtype=int) 
    
    for row in range(1, m + 1):
        D[row,0] = D[row - 1, 0] + del_cost
        
    for col in range(1, n + 1):
        D[0,col] = D[0, col - 1] + ins_cost
        
    for row in range(1, m + 1): 
        for col in range(1, n + 1):
            r_cost = rep_cost
            
            if source[row - 1] == target[col - 1]:
                r_cost = 0
            
            D[row,col] = min([D[row - 1, col] + del_cost, D[row, col - 1] + ins_cost, D[row - 1, col - 1] + r_cost])
          
    med = D[m, n]

    return D, med

In [36]:
source =  'what'
target = 'waht'
matrix, min_edits = min_edit_distance(source, target)
print("minimum edits: ",min_edits, "\n")
idx = list('#' + source)
cols = list('#' + target)
df = pd.DataFrame(matrix, index=idx, columns= cols)
print(df)

minimum edits:  2 

   #  w  a  h  t
#  0  1  2  3  4
w  1  0  1  2  3
h  2  1  2  1  2
a  3  2  1  2  3
t  4  3  2  3  2


In [37]:
source =  'eer'
target = 'deer'
matrix, min_edits = min_edit_distance(source, target)
print("minimum edits: ",min_edits, "\n")
idx = list(source)
idx.insert(0, '#')
cols = list(target)
cols.insert(0, '#')
df = pd.DataFrame(matrix, index=idx, columns= cols)
print(df)

minimum edits:  1 

   #  d  e  e  r
#  0  1  2  3  4
e  1  2  1  2  3
e  2  3  2  1  2
r  3  4  3  2  1


In [38]:
source = "eer"
targets = edit_one_letter(source,allow_switches = False)
for t in targets:
    _, min_edits = min_edit_distance(source, t,1,1,1)
    if min_edits != 1: print(source, t, min_edits)

In [39]:
source = "eer"
targets = edit_two_letters(source,allow_switches = False)
for t in targets:
    _, min_edits = min_edit_distance(source, t,1,1,1)
    if min_edits != 2 and min_edits != 1: print(source, t, min_edits)

eer eer 0
